In [9]:
#!pip install PyYAML
#!pip install ua-parser
#!pip install user-agents
!pip install fake-useragent

  Stored in directory: C:\Users\Mark\AppData\Local\pip\Cache\wheels\5e\63\09\d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


### 1. (10 points) Change the variable my_path to reflect a directory on your laptop where you will store the crawled websites and enter in ‘qmss columbia’ as the variable called ‘the_query’, then run the program (with my_top.py open run)
1. Perform the same search using googles search engine
2. Perform a quick comparison between the google crawl results and the outputted text files. What are some observations?

##### *1.2. The crawl results provide the urls that one could leverage to get to the websites associated with a "qmss columbia" google search. While the outputted txt files provide the scraped text found on each website. So if the crawler output is: http://qmss.squarespace.com/ ; then the associated txt file will contain the text from that website.*

In [13]:
class crawler(object):

    def my_scraper(self, tmp_url_in):
        from bs4 import BeautifulSoup
        import requests
        import re
        tmp_text = ''
        try:
            content = requests.get(tmp_url_in)
            soup = BeautifulSoup(content.text, 'html.parser')
    
            tmp_text = soup.findAll('p') 
    
            tmp_text = [word.text for word in tmp_text]
            tmp_text = ' '.join(tmp_text)
            tmp_text = re.sub('\W+', ' ', re.sub('xa0', ' ', tmp_text))
        except:
            pass
    
        return tmp_text
    
    def fetch_urls(self, query_tmp, cnt):
        #now lets use the following function that returns
        #URLs from an arbitrary regex crawl form google
    
        #pip install pyyaml ua-parser user-agents fake-useragent
        import requests
        from fake_useragent import UserAgent
        from bs4 import BeautifulSoup
        import re 
        ua = UserAgent()
    
        query = '+'.join(query_tmp.split())
        google_url = "https://www.google.com/search?q=" + query + "&num=" + str(cnt)
        print (google_url)
        response = requests.get(google_url, {"User-Agent": ua.random})
        soup = BeautifulSoup(response.text, "html.parser")

        result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})

        links = []
        titles = []
        descriptions = []
        for r in result_div:
            # Checks if each element is present, else, raise exception
            try:
                link = r.find('a', href = True)
                title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
                description = r.find('div', attrs={'class':'s3v9rd'}).get_text()
    
                # Check to make sure everything is present before appending
                if link != '' and title != '' and description != '': 
                    links.append(link['href'])
                    titles.append(title)
                    descriptions.append(description)
            # Next loop if one element is not present
            except:
                continue  
    
        to_remove = []
        clean_links = []
        for i, l in enumerate(links):
            clean = re.search('\/url\?q\=(.*)\&sa',l)
    
            # Anything that doesn't fit the above pattern will be removed
            if clean is None:
                to_remove.append(i)
                continue
            clean_links.append(clean.group(1))

        return clean_links
 
    def write_crawl_results(self, the_path, my_query, the_cnt_in):
        #let use fetch_urls to get URLs then pass to the my_scraper function 
        import os
        import re
        
        the_urls_list = self.fetch_urls(my_query, the_cnt_in)
        try:
            os.makedirs(the_path + re.sub('[ ]+', '_', re.sub('"', '', my_query)))
        except:
            pass

        cnt = 0
        for word in the_urls_list:
            tmp_txt = self.my_scraper(word)
            if len(tmp_txt) != 0:
                try:
                    tmp_file = open(the_path + re.sub('[ ]+', '_', re.sub('"', '', my_query)) + '/' + str(cnt) + '.txt',"w") 
                    tmp_file.write(tmp_txt)
                    tmp_file.close()
                    print (word)
                    cnt += 1
                except:
                    pass

In [1]:
from crawler import crawler

my_path = 'D:/QMSS/Spring/NaturalLanguageProcessing/Homework/HW2/crawled_websites_'
the_query = 'qmss columbia'
num_docs = 50

my_func = crawler()

my_func.write_crawl_results(my_path, the_query, num_docs)

https://www.google.com/search?q=qmss+columbia&num=50
http://www.qmss.columbia.edu/
https://gsas.columbia.edu/degree-programs/ma-programs/quantitative-methods-social-sciences
http://iserp.columbia.edu/hiring-payroll/qmss-ra-program
https://medium.com/%40RobinCRLee/review-on-columbia-qmss-ma-program-d22c0db2f0d7
https://www.facebook.com/Quantitative-Methods-in-Social-Sciences-QMSS-Columbia-University-147741968642064/
https://twitter.com/qmss_columbia%3Flang%3Den
https://www.goodreads.com/group/show/80365-qmss-at-columbia-university
https://economics.rice.edu/undergraduate-program/resources/student-opportunities/qmss-columbia-university
https://forum.thegradcafe.com/topic/121708-masters-programs-similar-to-columbia-qmss/
http://qmss.squarespace.com/
https://asintunado.wordpress.com/2016/10/02/m-a-quantitative-methods-in-the-social-sciences-at-columbia-university-things-you-should-know/
https://rdrr.io/github/jgabry/QMSS_package/man/QMSS.html
https://www.coursicle.com/columbia/courses/QMSS

### 2. (35 points) Refactor the code to loop through 4 topics of your choice; do not run the code yet, move to step 3.  Hint: the_query will need to be an array of 4 possible queries that you loop through

In [181]:
my_path = 'D:/QMSS/Spring/NaturalLanguageProcessing/Homework/HW2/crawled_websites_'

ques = ["new york mets", "tesla", "how to fry an egg", "sean pablo"]

def get_ques(var):
    for v in var:
        num_docs = 50 
        my_func = crawler()
        my_func.write_crawl_results(my_path, v, num_docs)
        
        



### 3. (55 points) Refactor the file called crawler.py to not write to text files, but rather append a pandas dataframe called ‘the_data’, accessible on my_top.py, that contains four columns:

1. 'body_basic' - basic removal of special characters; already coded for you

2. 'body_stem' - perform stemming (Porter) on body_basic and write resultant to this column

3. 'label' - what query the body relates to; if more than one word exist in the query, concatenate all words together with a ‘_’ between them all

4. Upload a zip file to CourseWorks with the refactored files

In [1]:
import pandas as pd

In [106]:
class crawler(object):

    def my_scraper(self, tmp_url_in):
        from bs4 import BeautifulSoup
        import requests
        import re
        tmp_text = ''
        try:
            content = requests.get(tmp_url_in)
            soup = BeautifulSoup(content.text, 'html.parser')
    
            tmp_text = soup.findAll('p') 
    
            tmp_text = [word.text for word in tmp_text]
            tmp_text = ' '.join(tmp_text)
            tmp_text = re.sub('\W+', ' ', re.sub('xa0', ' ', tmp_text))
        except:
            pass
    
        return tmp_text
    
    def fetch_urls(self, query_tmp, cnt):
        #now lets use the following function that returns
        #URLs from an arbitrary regex crawl form google
    
        #pip install pyyaml ua-parser user-agents fake-useragent
        import requests
        from fake_useragent import UserAgent
        from bs4 import BeautifulSoup
        import re 
        ua = UserAgent()
    
        query = '+'.join(query_tmp.split())
        google_url = "https://www.google.com/search?q=" + query + "&num=" + str(cnt)
        #print (google_url)
        response = requests.get(google_url, {"User-Agent": ua.random})
        soup = BeautifulSoup(response.text, "html.parser")

        result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})

        links = []
        titles = []
        descriptions = []
        for r in result_div:
            # Checks if each element is present, else, raise exception
            try:
                link = r.find('a', href = True)
                title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
                description = r.find('div', attrs={'class':'s3v9rd'}).get_text()
    
                # Check to make sure everything is present before appending
                if link != '' and title != '' and description != '': 
                    links.append(link['href'])
                    titles.append(title)
                    descriptions.append(description)
            # Next loop if one element is not present
            except:
                continue  
    
        to_remove = []
        clean_links = []
        for i, l in enumerate(links):
            clean = re.search('\/url\?q\=(.*)\&sa',l)
    
            # Anything that doesn't fit the above pattern will be removed
            if clean is None:
                to_remove.append(i)
                continue
            clean_links.append(clean.group(1))

        return clean_links
 
    def write_crawl_results(self, the_path, my_query, the_cnt_in):
        #let use fetch_urls to get URLs then pass to the my_scraper function 
        import os
        import re
        
        the_urls_list = self.fetch_urls(my_query, the_cnt_in)
        try:
            os.makedirs(the_path + re.sub('[ ]+', '_', re.sub('"', '', my_query)))
        except:
            pass

        cnt = 0
        
        for word in the_urls_list:
            tmp_txt = self.my_scraper(word)        
            if len(tmp_txt) != 0:
                try:
                    tmp_file = open(the_path + re.sub('[ ]+', '_', re.sub('"', '', my_query)) + '/' + str(cnt) + '.txt',"w") 
                    tmp_file.write(tmp_txt)
                    tmp_file.close()
                    print (word)
                    cnt += 1
                except:
                    pass
                
                
   

In [114]:
my_path = 'D:/QMSS/Spring/NaturalLanguageProcessing/Homework/HW2/Crawled/'

ques = ["new york mets", "tesla", "how to fry an egg", "sean pablo"]

def get_ques(var):
    for v in var:
        num_docs = 50
        my_func = crawler()
        my_func.write_crawl_results(my_path, v, num_docs)#v = the_query
        
        


In [ ]:
get_ques(ques)

https://twitter.com/Mets/status/1233520503324893186%3Fref_src%3Dtwsrc%255Egoogle%257Ctwcamp%255Eserp%257Ctwgr%255Etweet
https://www.mlb.com/mets
https://www.espn.com/mlb/team/_/name/nym/new-york-mets
https://en.wikipedia.org/wiki/New_York_Mets
https://bleacherreport.com/new-york-mets
https://www.sny.tv/mets/news/
https://nypost.com/2020/02/28/how-mets-legend-dwight-gooden-stopped-sign-stealing/
https://www.baseball-reference.com/teams/NYM/index.shtml
https://www.youtube.com/channel/UCgIMbGazP0uBDy9JVCqBUaA
https://www.cbssports.com/mlb/teams/NYM/new-york-mets/
https://www.foxnews.com/sports/mets-brodie-van-wagenen-venmo-fans
https://www.foxsports.com/mlb/new-york-mets-team
https://www.mlbtraderumors.com/new-york-mets
https://www.sbnation.com/mlb/teams/new-york-mets
https://www.stubhub.com/new-york-mets-tickets/performer/5649/
https://www.mlbshop.com/new-york-mets/t-36556586%2Bz-9173839-345902959
https://www.amazinavenue.com/
https://www.tmz.com/2020/02/27/new-york-mets-grant-wish-fan-c

In [109]:
import pandas as pd
import re
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 

In [110]:
my_stem = PorterStemmer()
stop_words = set(stopwords.words('english'))
stop_words.add('e')

In [111]:
 def clean_data(var, the_path_in, dir_name_in, stop_words_in):
        f = open(the_path_in + dir_name_in + '/' + var, "r",
                 encoding='ISO-8859-1')
        tmp_read = re.sub('[^a-zA-Z]+', ' ', str(f.read()))
        f.close()
        
        g = open(the_path_in + dir_name_in + '/' + var, "r",
                 encoding='ISO-8859-1')
        tmp_read2 = str(g.read())
        g.close()
        
        tmp_read = [word.lower() for word in tmp_read.split() if word not in stop_words_in]
        tmp_read = [my_stem.stem(word) for word in tmp_read]
        tmp_read = ' '.join(tmp_read)
        tmp_read2 = [word for word in tmp_read2.split()]
        tmp_read2 = ' '.join(tmp_read2)
        return tmp_read,tmp_read2

In [112]:
the_path = 'D:/QMSS/Spring/NaturalLanguageProcessing/Homework/HW2/Crawled/'

the_dirs = os.listdir(the_path)
the_df = pd.DataFrame()
for dir_name in the_dirs:
    the_filenames = os.listdir(the_path + dir_name)
    for word in the_filenames:
        tmp_read = clean_data(word, the_path, dir_name, stop_words)
        tmp = pd.DataFrame([tmp_read], columns=['body_stem', 'basic_body'])
        tmp['label'] = dir_name
        the_df = the_df.append(tmp, ignore_index=True)
        



In [119]:
the_df.head(50)

,body_stem,basic_body,label
0,these step step tip teach make fri egg whether...,These step by step tips will teach you how to ...,how_to_fry_an_egg
1,delish editor handpick everi product featur we...,Delish editors handpick every product we featu...,how_to_fry_an_egg
2,we experienc problem retriev page request inci...,We ve experienced a problem retrieving the pag...,how_to_fry_an_egg
3,ordinarili hear merit fri egg base prefer fri ...,Ordinarily you hear about the merits of fried ...,how_to_fry_an_egg
4,felic cloak wed nov est first publish wed nov ...,Felicity Cloake Wed 7 Nov 2012 19 10 EST First...,how_to_fry_an_egg
5,stump dinner get life save dinner daili newsle...,Stumped for dinner Get our life saving Dinner ...,how_to_fry_an_egg
6,flip fri egg one intimid act beginn breakfast ...,Flipping your fried eggs is one of the most in...,how_to_fry_an_egg
7,for better experi delia onlin websit enabl jav...,For a better experience on Delia Online websit...,how_to_fry_an_egg
8,easi add littl oil fri pan crack egg rounder s...,Easy Add a little oil to a frying pan and crac...,how_to_fry_an_egg
9,my favorit way egg fri i often coupl time week...,My favorite way of having eggs is fried I don ...,how_to_fry_an_egg
